# Training & Deploying TensorFlow Models at Scale

Once you have a beautiful model that makes amazing predictions, what do you do with it? Well, you need to put it in production! This could be as simple as running the model on a batch of data & perhaps writing a script that runs the model every night. However, it is often much more involved. Various parts of your infrastructure may need to use this model on live data, in which case you probably want to wrap your model in a web service: this way, any part of your infrastructure can query your model at any time using a simple REST API (or some other protocol). But as time passes, you need to regularly retrain your model on fresh gracefully transition from one model to the next, possibly roll back to the previous model in case of problems, & perhaps run multiple different models in parallel to perform *A/B experiments*. If your product becomes successful, your service may start to get plenty of *queries per second* (QPS), & it must scale up to support the load. A great solution to scale up your service, as we will see in this lesson, is to use TF serving, either on your own hardware infrastructure or via a cloud service such as Google Cloud AI Playform. It will take care of efficiently serving your model, handle graceful model transitions, & more. If you use the cloud platform, you will also get many extra features, such as powerful monitoring tools.

Moreover, if you have a lot fo training data, & compute-intensive models, then training time may be prohibitively long. If your product needs to adapt to changes quickly, then a long training time can be a showstopper (e.g., think of a news recommendation system promoting news from last week). Perhaps even more importantly, a long training time will prevent you from experimenting with new ideas. In machine learning (as in many other fields), it is hard to know in advance which ideas will work, so you should try out as many as possible, as fast as possible. One way to speed up training is to use hardware accelerators such as GPUs or TPUs. To go even faster, you can train a model across multiple machines, each equipped with multiple hardware accelerators. TensorFlow's simple yet powerful distribution strategies API makes this easy, as we will see.

In this lesson, we will look at how to deploy models, first to TF serving, then to Google Cloud AI Platform, we will also take a quick look at deploying models to modile apps, embedded devices, & web apps. Lastly, we will discuss how to speed up computations using GPUs & how to train models across multiple devices & servers using the distribution strategies API. That's a lot of topics to discuss, so let's get started.

---

# Serving a TensorFlow Model

Once you have trained a TensorFlow model, you can easily use it in any python code: if it's a tf.keras model, just call its `predict()` method! But as your infrastructure grows, there comes a point where it is preferable to wrap your model in a small service whose sole role is to make predictions & have the rest of the infrastructure query it (e.g., via a REST or gRPC API). This decouples your model from the rest of the infrastructure, making is possible to easily switch model versions or scale the service up as needed (independently from the rest of your infrastructure), perform A/B experiements, & ensure that all your software components rely on the same model versions. It also simplifies testing & development, & more. You can create your own microservice using any technology you want (e.g., using the flask library), but why reinvent the wheel when you can just use TF serving?

## Using TensorFlow Serving

TF serving is very efficient, battle-tested model server that's written in C++. It can sustain a high load, serve multiple versions of your models & watch a model repository to automatically deploy the latest versions & more.

<img src = "Images/TF Serving.png" width = "550" style = "margin:auto"/>

So let's suppose you have trained an MNIST model using tf.keras, & you want to deploy it to TF serving. The first thing you have to do is export this model to TensorFlow's *SavedModel format*.

### Exporting SavedModels

TensorFlow provides a simple `tf.saved_model.save()` function to export models to the SavedModel format. All you need to do is give it the model, specifying its name & version number, & the function will save the model's computation graph & its weights:

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

(X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()
X_train = X_train[..., np.newaxis].astype(np.float32) / 255.0
X_test = X_test[..., np.newaxis].astype(np.float32) / 255.0
X_val, X_train = X_train[:5000], X_train[5000:]
y_val, y_train = y_train[:5000], y_train[5000:]
X_new = X_test[:3]

In [ ]:
keras.backend.clear_session()

model = keras.models.Sequential([
    keras.layers.Input(shape = [28, 28, 1]),
    keras.layers.Flatten(),
    keras.layers.Dense(300, activation = "relu"),
    keras.layers.Dense(100, activation = "relu"),
    keras.layers.Dense(10, activation = "softmax")
])
model.compile(loss = "sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(learning_rate = 1e-2),
              metrics = ["accuracy"])
model.fit(X_train, y_train, epochs = 20, validation_data = (X_val, y_val))

In [ ]:
import os

model_version = "001"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
tf.saved_model.save(model, model_path)

In [ ]:
#model.save(model_path)

You can also just use the model's `save()` method (`model.save(model_path)`): as long as the file's extension is not `.h5`, the model will be saved using a SavedModel format instead of the HDF5 format.

It's usually a good idea to include all the preprocessing layers in the final model you export so that it can ingest data in its natural form once it is deployed to production. This avoids having to take care of preprocessing separately within the application that uses the model. Bundling the preprocessing steps within the model also makes it simpler to update them later on & limits the risk of mismatch between a model & the preprocessing steps it requires.

A SavedModel represents a version of your model. It is stored as a directory containing a *saved_model.pb* file, which defines the computation graph (represented as a serialised protocol buffer), & a *variables* subdirectory containing the variable values. For models containing a large number of weights, these variable values may be split across multiple files. A SavedModel also includes an *assets* subdirectory that my contain additional data, such as vocabulary files, class names, or some example instances for this model. The directory structure is as follows (in this example, we don't use assets):

* `my_mnist_model`
   - `001`
      * `assets`
      * `saved_model.qb`
      * `variables`
         - `variables.data-00000-of-00001`
         - `variables.index`
       
As you might expect, you can load a SavedModel using the `tf.saved_model.load()` function. However, the returned object is not a keras model: it represents the SavedModel, including its computation graph & variable values. You can use it like a function, & it will make predictions (make sure to pass the inputs as tensors of the appropriate type):

In [ ]:
saved_model = tf.saved_model.load(model_path)
y_pred = saved_model(tf.constant(X_new, dtype = tf.float32))

Alternatively, you can load this SavedModel directly to a keras model using the `keras.models.load_model()` function:

In [ ]:
model = keras.models.load_model(model_path)
y_pred = model.predict(tf.constant(X_new, dtype = tf.float32))

TensorFlow also comes with a small `saved_model_cli` command-line tool to inspect SavedModels:

In [ ]:
!saved_model_cli show --dir {model_path}

In [ ]:
!saved_model_cli show --dir {model_path} --tag_set serve

In [ ]:
!saved_model_cli show --dir {model_path} --tag_set serve \
                      --signature_def serving_default

A SavedModel contains one or more *metagraphs*. A metagraph is a computation graph plus some function signature definitions (including their input & output names, types, & shapes). Each metagraph is identified by a set of tags. For example, you may want to have a metagraph containing the full computation graph, including the training operations (this one may be tagged `"train"`, for example), & another metagraph containing a pruned computation graph with only the prediction operations, including some GPU-specific operations 9this metagraph may be tagged `"serve"`, `"gpu"`). However, when you pass a tf.keras. model to the `tf.saved_model.save()` function, by default the function saves a much simpler SavedModel: it saves a single metagraph tagged `"serve"`, which contains two signature definitions, an initialisation function (called `__saved_model_init_op`, which you do not need to worry about) & a default serving function (called `serving_default`). When saving a tf.keras model, the default serving function corresponds to the model's `call()` function, which of course makes predictions.

The `saved_model_cli` tool can also be used to make predictions (for testing, not really for production). Suppose you have a numpy array (`X_new`) containing three images of handwritten digits that you want to make predictions for. You first need to export them to numpy's `npy` format:

In [ ]:
np.save("my_mnist_tests.npy", X_new)

Next, use the `save_model_cli` command like so:

In [ ]:
!saved_model_cli run --dir {model_path} --tag_set serve \
                     --signature_def serving_default    \
                     --inputs {input_name}=my_mnist_tests.npy

The tool's output contains 10 class probabilities for each of the 3 instances. Great! Now that you have a working SavedModel, the next step is to install TF serving.

### Installing TensorFlow Serving

There are many ways to install TF serving: using a docker image, using a system's package manager, installing from source, & more. Let's use the docker option, which is highly recommended by the TensorFlow team as it is simple to install, it will not mess with your system, & it offers high performance. You first need to install docker. Then download the offical TF serving docker image:

```
docker pull tensorflow/serving
```

Now you can create a docker container to run this image:

```
docker run -it --rm -p 8500:8500 -p 8501:8501 \
            -v "$ML_PATH/my_mnist_model:/models/my_mnist_model" \
            -e MODEL_NAME=my_mnist_model \
            tensorflow/serving
```

That's it! TF serving is running. It loaded our MNIST model (version 1), & it is serving it through both gRPC (on port 8500) & REST (on port 8501). Here is what all the command-line options mean:

- `-it`
   - Makes the container interactive (so you can press Ctrl-C to stop it) & displays the sever's output.
- `-rm`
   - Deletes the container when you stop it (no need to clutter your machine with interrupted containers). However, it does not delete the image.
- `-p 8500:8500`
   - Makes the docker engine forward the host's TCP port 8500 to the container's TCP port 8500. By default, tf serving uses this port to serve the gRPC API.
- `-p 8501:8501`
   - Forwards the host's TCP port 8501 to the container's TCP port 8501. By default, TF Serving uses this port to serve the REST API.
- `-v "$ML_PATH/my_mnist_model:/models/my_mnist_model"`
   - Makes the host's `$ML_Path/my_mnist_model` directory available to the container at the path */models/mnist_model*. On windows, you may need to replace / with \ int he host path (but not in the container path).
- `-e MODEL_NAME=my_mnist_model`
   - Sets the container's `MODEL_NAME` environment variable, so TF serving knows which model to serve. By default, it will look for models in the /*models* directory, & it will automatically serve the latest verion it finds.
- `tensorflow_serving`
   - This is the name of the image to run.

Now let's go back to Python & query this server, first using the REST API, then the gRPC API.

### Querying TF Serving Through the REST API

Let's start by creating the query. It must contain the name of the function signature you want to call, & of course the input data:

In [ ]:
import json

input_dada_json = json.dumps({"signature_name": "serving_default",
                              "instances": X_new.tolist()})

Note that the JSON format is 100% text-based, so the `X_new` numpy array had to be converted to a python list & then formatted as JSON:

In [ ]:
input_data_json

Now let's send the input data to TF serving by sending an HTTP POST request. This can be done easily using the `requests` library (it is not part of python's standard library, so you will need to install it first, e.g., using pip):

In [ ]:
import requests

server_url = "http://localhost:8501/v1/models/my_mnist_model:predict"
response = requests.post(server_url, data = input_data_json)
response.raise_for_status()
response = response.json()

The response is a dictionary containing a single `"predictions"` key. The corresponding value is the list of predictions. The list is a python list, so let's convert it to a numpy array & round the floats it contains to the second ddecimal:

In [ ]:
y_proba = np.array(response["predictions"])
y_proba.round(2)

Hooray! We have the predictions. The model is close to 100% confident that the first image is a 7, 99% confident the second image is a 2, & 96% confident that the third image is a 1.

The REST API is nice & simple, & it works well when the input & output data are not too large. Moreover, just about any client application can make REST queries without additional dependencies, whereas other protocols are not always so readily available. However, it is based on JSON, which is text-based & fairly verbose. For example, we had to convert the numpy array to a python list, & every float ended up represented as a string. This is very inefficient, both in terms of serialisation/deserialisation time (to convert all the floats to strings & back) & in terms of payload size: many floats end up being represented using over 15 characters, which translates to over 120 bits for 32-bit floats! This will result in high latency & bandwidth usage when transferring large numpy arrays. So let's use gRPC instead.

### Querying TF Serving Through The gRPC API

The gTPC API expects a serialised `PredictRequest` protocol buffer as input, & it outputs a serialised `PredictResponse` protocol buffer. These protobufs are part of the `tensorflow-serving api` library, which you must install (e.g., using pip). First, let's create the request:

In [ ]:
from tensorflow_serving.apis.predict_pb2 import PredictRequest

request = PredictRequest()
request.model_spec.name = model_name
request.model_spec.signature_name = "serving_default"
input_name = model.input_names[0]
request.inputs[input_name].CopyFrom(tf.make_tensor_proto(X_new))

This code creates a `PredictRequest` protocol buffer & fills in the required fields, including the model name, the signature name of the function we want to call, & finally the input data, in the form of a `Tensor` protocol buffer. The `tf.make_tensor_proto()` function creates a `Tensor` protocol buffer based on the fiven tensor or numpy array, in this case `X_new`.

Next, we'll send the request to the serve & get its response (for this you will need the `grpcio` library, which you can install using pip):

In [ ]:
import grpc
from tensorflow_serving.apis import prediction_service_pb2_grpc

channel = grpc.insecure_channel("localhost:8500")
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
response = predict_service.Predict(request, timeout = 10.0)

The code is quite straightforward: after the imports, we create a gRPC communication channel to *localhost* to TCP port 8500, then we create a gRPC service over this channel & use it to sebd a requestm with a 10-second timeout (not that the call is synchronous: it will block until it receives the response or the timeout period expires). In this example, the channel is insecure (no encryption, no authentication), but gRPC & TensorFlow serving also support secure channels over SSL/TLS.

Next, let's convert the `PredictResponse` protocol buffer to a tensor:

In [ ]:
output_name = model.output_names[0]
predict_service = prediction_service_pb2_grpc.PredictionServiceStub(channel)
y_proba = tf.make_ndarray(outputs_proto)

If you run this code & print `y_proba.numpy().round(2)`, you will get the exact same estimated class probabilities as earlier. That's all there is to it: in just a few lines of code, you can now access your TensorFlow model remotely, using either REST or gRPC.

### Deploying A New Model Version

Now let's create a new model version & export a SavedModel to the *my_mnist_model/002* directory, just like earlier:

In [ ]:
keras.backend.clear_session()

model2 = keras.models.Sequential([
    keras.layers.Input(shape = [28, 28, 1]),
    keras.layers.Flatten(),
    keras.layers.Dense(100, activation = "relu"),
    keras.layers.Dense(10, activation = "softmax")
])
model2.compile(loss = "sparse_categorical_crossentropy",
              optimizer = keras.optimizers.SGD(learning_rate = 1e-2),
              metrics = ["accuracy"])
model2.fit(X_train, y_train, epochs = 20, validation_data = (X_val, y_val))

In [ ]:
model_version = "002"
model_name = "my_mnist_model"
model_path = os.path.join(model_name, model_version)
tf.saved_model.save(model2, model_path)

At regular intervals (the delay is configurable), TensorFlow serving checks for new model versions. If it finds one, it will automatically handle the transition gracefully: by default, it will answer pending requests (if any) with the previous model version, while handling new requests with the new version. As soon as every pending request has been answered, the previous model version is unloaded. You can see this at work in the TensorFlow serving logs.

This approach offers a smooth transition, but it may use too much RAM (especially GPU RAM, which is generally the most limited). In this case, you can configure TF Serving so that it handles all pending requests with the previous model version & unloads it before loading & using the new model version. This configuration will avoid having two model versions loaded at the same time, but the service will be unavailable for a short period.

As you can see, TF serving makes it quite simple to deploy new models. Moreover, if you discover that version 2 does not work as well as you expected, then rolling back to version 1 is as simple as removing the *my_mnist_model/002* directory.

If you expect to get many queries per second, you will want to deploy TF serving on multiple servers & load-balance the queries. This will require deploying & managing many TF serving containers across their servers. One way to handle that is to use a tool such as Kubernetes, which is an open source system for simplifying container orchestration across many servers. If you do not want to purchase, maintain, & upgrade all the hardware infrastructure, you will want to use virtual machines on a cloud platform such as Amazon AWS, Microsoft Azure, Google Cloud Platform, IBM Cloud, Alibaba Cloud, Oracle Cloud, or some other Platform-as-a-Service (PaaS). managing all the virtual machines, handling container orchestration (even with the help of Kubernetes), taking care of TF Serving configuration, tuning & monitoring -- all of this can be a full-time job. Fortunately, some service providers can take care of all this for you. In this lesson, we will use Google Cloud AI Platofrm because it's the only platofrm with TPUs today, it supports TensorFlow 2, & it offers a nice suite of AI services (e.g., AutoML, Vision API, Natural Language API). There are several other providers in this space, such as Amazon AWS SageMaker & Microsoft AI Platform, which are also capable of serving TensorFlow models.

<img src = "Images/Scale Up TF Serving with Load Balancing.png" width = "500" style = "margin:auto"/>

Now let's see how to serve our wonderful MNIST model on the cloud!

## Creating a Prediction Service on GCP AI Platform

Before you can deploy a model, there's a little bit of setup to take care of:

1. Log in your Google account, then go to the Google Cloud Platform (GCP) console. If you don't have a Google account, you'll have to create one.
2. If it is your first time using GCP, you will have to read & accept the terms & conditions. Click Tour Console if you want. At the time of this writing, new users are offered a free trial, including $300 worht of GCP credit that youc an use over the course of 12 months. You will only need a small portion of that to pay for the services you will need in this lesson. Upon signing up for the free trial, you will still need to create a payment profile & enter your credit card number: it is used for verification purposes (probably to avoid people using the free trial multiple times), but you will not be billed. Activation & upgrade your account if requested.

<img src = "Images/Google Cloud Platform.png" width = "600">

3. If you have used GCP before & your free trial has expired, then the services you will use in this chapter will cost you some money. It should not be much, especially if you remember to turn off the services when you don't need them anymore. Make sure you understand & agree to the pricing conditions before you run any service. Make sure your billing acount is active. To check, open the navigation menu on the left & click Billing, & make sure you have set up a pyment method & that the billing account is active.
4. Every resource in GCP belongs to a project. This includes all the virtual machines you may use, the files you store, & the training jobs you run. When you create an account, GCP automatically creates a project for you called "My First Project". If you want, you can change its display nameby going to the project settings: in the navigation menu (on the left of the screen), select IAM & admin -> Settings, change the project's display name, & click Save. Note that the project also has a unique ID & number. You can choose the project ID when you create a project, but you cannot change it later. The project number is automatically generated & cannot be changed. If you want to create a new project, click the project name at the top of the page, then click New Project & enter the project ID. Make sure billing is active for this new project.
5. Now that you have a GCP account with billing activated, you can start using the services. The first one you will need is Google Cloud Storage (GCS): this is where you will put the SavedModels, the training data, & more. In the navigation menu, scroll down to the Storage section, click Storage -> Browser. All your files will go in one or more *buckets*. Click Create Bucket & choose the bucket name (you may need to activation the Storage API first). GCS uses a single worldwide namespace for buckets, so simple names like "machine-learning" will most likely not be available. Make sure the bucket name conforms to the DNS naming conventions, as it may be used in DNS records. Moreover, bucket names are public, so do not put anything private in there. It is common to use your domain name or your company name as prefix to ensure uniqueness, or simple use a random as a part of the name. Choose the location where you want the bucket to be hosted, & the rest of the options should be fine by default. Then click Create.
6. Upload the *my_mnist_model* folder you created earlier (included one or more versions) to your bucket. To do this, just go to the GCS Browser, click the bucket, then drag & drop the *my_mnist_model* folder from your system to the bucket. Alternatively, you can clikc "Upload folder" & select the *my_mnist_model* folder to upload. By default, the maximum size for a SavedModel is 250 MB, but it is possible to request a higher quota.

<img src = "Images/Uploading SavedModel to Google Cloud Storage.png" width = "600" style = "margin:auto"/>

7. Now you need to configure AI Platform (formerly known as ML Engine) so that it knows which models & versions you want to use. In the navigation menu, scroll down to the Artificial Intelligence section, & click AI Platform -> Models. Click Activate API (it takes a few minutes), then click "Create model". Fill in the model deatils & click Create.

<img src = "Images/Creating a New Model on Google Cloud AI Platform.png" width = "600" style = "margin:auto"/>

8. Now that you have a model on AI Platform, you need to create a model version. In the list of models, click the model you just created, then click "Create version" & fill in the version details: set the name, description, python version (3.5 or above), framework (TensorFlow), framework version (2.0 if available or 1.13), ML runtime version (2.0, if available or 1.13), machine type (choose "Single Core CPU" for now), model path on GCS (this is the full path to the actual version folder, e.g., (gs://my-mnist-model-bucket/my_mnist_model/002/*), scaling (choose automatic), & minimum number of TF serving containers to have running at all times (leave this fields empty). Then click Save.

<img src = "Images/Creating a New Model Version on Google Cloud AI Platform.png" width = "600" style = "margin:auto"/>

Congratulations, you have deployed your first model on the cloud! Because you selected automatic scaling, AI Platform will start more TF serving containers when the number of queries per second increases, & it will load-balance the queries between them. If the QPS goes down, it will stop containers automatically. The cost is therefore directly linked to the QPS (as well as the type of machine you choose & the amount of data you store on GCS). This pricing model is particularly useful for occasional users & for services with important usage spikes, as well as for startups: the price remains low until the startup actually starts up.

Now let's query this prediction service!

## Using the Prediction Service

Under the hood, AI Platform just runs TF serving, so in principle you could use the same code as earlier, if you knew which URL to query. There's just one problem: GCP also takes care of encryption & authentication. Encryption is based on SSL/TLS, & authetication is token-based: a secret authentication token must be sent to the server in every request. So before your code can use the prediction service (or any other GCP service), it must obtain a token. We will see how to do this shortly, but first you need to configure authetication & give your application the appropriate access rights on GCP. You have two options for authentication:

* Your application (i.e., the client code that will query the prediction service) could authenticate using user credentials with your own Google login & password. Using user credentials would give your application the exact smae rights as on GCP, which is certainly way more than it needs. Moreover, you would have to deploy your credentials in your application, so anyone with access could steal your credentials & fully access your GCP account. In short, do not choose this option; it is only needed in very rare cases (e.g., when your application needs to access its user's GCP account).
* The client code can authenticate with a *service account*. This is an account that represents an application, not a user. It is generally given very restricted access rights: strictly what it needs, & no more. This is the recommended option.

So, let's create a service account for your application: in the navigation menu, go to IAM & admin -> Service accounts, then click Create Service Account, fill in the form (service account name, ID, description), & click Create. Next, you must give this account some access rights. Select the ML Engine Developer role: this will allow the service account to make predictions, & not much more. Optionally, you can grant some users access to the service account (this is useful when your GCP user account is part of an organisation, & you wish to authorise other users in the organisation to deploy applications that will be based on this service account or to manage the service account itself). Next, click Create Key to expore the service account's private key, choose JSON, & click Create. This will download the private key in the form of a JSON file. Make sure to keep it private!

<img src = "Images/Create New Service Account in Google IAM.png" width = "600" style = "margin:auto"/>

Great! Now let's write a small script that will query the prediction service. Google provides several libraries to simplify access to its services:

* *Google API Client Library*
   - This is a fairly thin layer on top of *OAuth 2.0* (for the authentication) & REST. You can use it with all GCP services, including AI Platform. You can install it using pip: the library is called `google-api-python-client`.
* *Google Cloud Client Libraries*
   - These are a bit more high-level: each one is dedicated to a particular service, such as GCS, Google BigQuery, Google Cloud Natural Language, & Google Cloud Vision. All these libraries can be installed using pip (e.g., the GCS Client Library is called `google-cloud-storage`). When a client library is available for a given service, it is recommended to use it rather than the Google API Client Library, as it implements all the best practices & will often use gRPC rather than REST for better performance.

We will use the Google API Client Library. It will need to use the service account's private key; you can tell it where it is by stting the `GOOGLE_APPLICATION_CREDENTIALS` environment variable, either before starting the script or within the script like this:

In [ ]:
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "my_service_account_key.json"

Next, you must create a resource object that wraps access to the prediction service:

In [ ]:
import googleapiclient.discovery

project_id = "onyx-smoke-242003"
model_id = "my_mnist_model
model_path = "projects/{}/models/{}".format(project_id, model_id)
ml_resources = googleapiclient.discovery.build("ml", "v1").projects()

Note that you can append `/versions/001` (or any other version number) to the `model_path` to specify the version you want to query: this can be useful for A/B testing or for testing a new version on a small group of users before releasing it widely (this is called a *canary*). Next, let's write a small function that will use the resource object to call the prediction service & get the predictions back:

In [ ]:
def predict(X):
    input_data_json = {"signature_name": "serving_default",
                       "instances": X.tolist()}
    request = ml_resource.predict(name = model_path, body = input_data_json)
    response = request.execute()
    if "error" in response:
        raise RuntimeError(response["error"])
    return np.array([pred[output_name] for pred in response["predictions"]])

The function takes a numpy array containing the input images & prepares a dictionary that the client library will convert to the JSON format (as we did earlier). Then it prepares a prediction request, & executes it; it raises an exception if the response contains an error, or else it extracts the predictions for each instance & bundles them in a numpy array. Let's see if it works:

In [ ]:
Y_probas = predict(X_new)
np.round(Y_probas, 2)

Yes! You now have a nice prediction service running on the cloud that can automatically scale up to any number of QPS, plus you can query it from anywhere securely. Moreover, it costs you close to nothing when you don't use it: you'll pay just a few cents per month per gigabyte used on GCS. You also get detailed logs & metrics using Google Stackdriver.

But what if you want to deploy your model to a mobile app? Or to an embedded device?

---

# Deploying a Model to a Mobile or Embedded Device

If you need to deploy your model to a model or embedded device, a large model may simply take too long to download & use too much RAM & CPU, all of which will make your app unresponsive, heat the device, & drain its battery. To avoid this, you need to make a mobile-friendly, lightweight, & efficient model, without sacrificing too much of its accuracy. The tflite library provides several tools to help you deploy your models to mobile & embedded devices, with three main objectives:

* Reduce the model size, to shorten download time & reduce RAM usage.
* Reduce the amount of computations needed for each prediction, to reduce latency, battery usage, & heating.
* Adapt the model to device-specific constraints.

To reduce the model size, tflite's model converter can take a SavedModel & compress it to a much lighter format based on FlatBuffers. This is an efficient crossplatform serialisation library (a bit like protocol buffers) initially created by Google for gaming. It is designed so you can load FlatBuffers straight to RAM without any preprocessing: this reduces the loading time & memory footprint. Once the model is loaded into a mobile or embedded device, the tflite interpreter will execute it to make predictions. Here is how you can convert a SavedModel to FlatBuffer & save it to a *.tflite* file:

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_path)
tflite_model = converter.convert()
with open("converted_model.tflite", "wb") as f:
    f.write(tflite_model)

The converter also optimises the model, both to shrink it & reduce its latency. It prunes all the operations that are not needed to make predictions (such as training operations), & it optimises computations whenever possible; for exampl, $3a + 4a + 5a$ will be converted to $(3 + 4 + 5)a$. It also tries to fuse operations whenever possible. For example, batch normalisation layers end up folder into the previous layer's addition & multiplication operations, whenever possible. To get a good idea of how much tflite can optimise a model, download one of the pretrained tflite models, unzip the archive, then open the excellent Netron graph visualisation tool & upload the *.pb* file to view the original model. It's a big complex graph, right? Next, open the optimised *.tflite* model & marvel at its beauty!

Another way you can reduce the model size (other than simply using smaller neural network architectures) is by using smaller bit-widths: for example, if you use half-floats (16 bits) rather than regular floats (32 bits), the model size will shrink by a factor of 2, at the cost of a (generally small) accuracy drop. Moreover, training will be faster, & you will use roughly half the amount of GPU RAM.

TFLite's converter can go further than that, by quantising the model weights down to fixed point, 8-bit integers! This leads to a fourfold size reduction compared to using 32-bit floats. The simplest approach is called *post-training quantisation*: it just quantises the weights after training, using a fairly basic but efficient symmetrical quantisation technique. It finds the maximum absolute weight value, *m*, then it maps the floating-point range *-m* to *+m* to the fixed-point (integer) range -127 to +127. For example, if the weights range from -1.5, 0.0, & +1.5, respectively. Note that 0.0 always maps to 0 when using symmetrical quantisation (also note that the byte values +68 to +127 will not be used, since they map to floats greater than +0.8).

<img src = "Images/32-Bit Floats to 8-Bit Integers Using Symmetrical Quantisation.png" width = "600" style = "margin:auto"/> 

To perform this post-training quantisation, simply add `OPTIMIZE_FOR_SIZE` to the list of converter optimisations before calling the `convert()` method:

In [ ]:
converter.optimisations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]

This technique dramatically reduces the model's size, so it's much faster to download & store. However, at runtime, the quantised weights get converted back to floats before they are used (these recovered floats are not perfectly identifcal to the original floats, but not too far off, so the accuracy loss is usually acceptable). To avoid recomputing them all the time, the recovered floats are cached, so there is no reduction of RAM usage. There is also no reduction either in compute speed.

The most effective way to reduce latency & power consumption is to also quantise the activations so that the computations can be done entirely with integers, without the need for any floating-point operations. Even when using the same bit-width (e.g., 32-bit integers instead of 32-bit floats), integer computations use less CPU cycles, consume less energy, & produce less heat. If you also reduce the bit-width (e.g., down to 8-bit integers), you can get huge speedups. Moreover, some neural network accelerator devices (such as the edge TPU) can only process integers, so full quantisation of both weights & activations is compulsory. This can be done post-training; it requires a calibration step to find the maximum absolute value of the activations, so you need to provide a representative sample of training data to tflite (it does not need to be huge), & it will process the data through the model & measure the activation statistics required for quantisation (this step is typically fast).

The main problem with quantisation is that it loses a bit of accuracy: it is equivalent to adding noise to the weights & activations, if the accuracy drop is too severe, then you may need to use *quantisation-aware training*. This means adding fake quantisation operations to the model so it can learn to ignore the quantisation noise during training; the final weights will then be more robust to quantisation. Moreover, the calibration step can be taken care of automatically during training, which simplifies the whole process.

We've explored the core concepts of tflite, but going all the way to coding a mobile app or an embedded program would require a whole other book. Fortunately, one exists: if you want to learn more about building TensorFlow applications for mobile or embedded devices, check out the book: *TinyML: Machine Learning with TensorFlow on Arduino & Ultra-Low Power Micro-Controllers*, by Pete Warden (who leads the tflite team) & Daniel Situnayake.

Next, we will see how to use GPUs to speed up computations.

---

# Using GPUs to Speed Up Computations

We've discussed several techniques that can considerably speed up training: better weight initialisation, batch normalisation, sophisticated optimisers, & so on. But even with all of these techniques, training a large neural network on a single machine with a single CPU can take days or even weeks. 

In this section, we will look at how to speed up your models using GPUs. We will also see how to split the computations across multiple devices, including teh CPU & multiple GPU devices.  For now, we will run everything on a single machine, but later, we will discuss how to distribute computations across multiple servers.

<img src = "Images/Execute TensorFlow Grpah Across Multiple Devices in Parallel.png" width = "600" style = "margin:auto"/>

Thanks to GPUs, instead of waiting for days or weeks for a training algorithm to complete, you mayend up waiting for just a few minutes or hours. Not only does this save an enormous amount of time, but it also means that you can experiment with various models much more easily & frequentily retrain your models on fresh data.

The first step is to get yourhands on a GPU. There are two options for this: you can either purchase your own GPU(s), or you can use GPU-equipped virtual machines on the cloud. Let's start with the first option.

## Getting Your Own GPU

If you choose to purchase a GPU card, then take some time to make the right choice. Tim Dettmers wrote an excellent blog post to help you choose, & he updates it regularly: I encourage you to read it carefully. At the time of this writing, TensorFlow only supports Nvidia cards with CUDA Compute Capability 3.5 + (as well as Google's TPUs, of course), but it may extend its support to other manufacturers. Moreover, although TPUs are currently only available on GCP, it is highly likely that TPU-like cards will be available for sail in the near future, & TensorFlow may support them. In short, make sure to check TensorFlow's documentation to see what devices are supported at this point.

IF you go for an Nvidia GPU card, you will need to install the appropriate Nvidia drivers & several Nvidia libraries. These include the *Compute Unified Device Architecture* library (CUDA), which allows developers to use CUDA-enabled GPUs for all sorts of computations (not just graphics acceleration), & the CUDA *Deep Neural Network* library (cuDNN), a GPU-accelerated library of primitives for DNNs. cuDNN provides optimised implementations of common DNN computations such as activation layers, normalisation, forward & backward convolutions, & pooling. It is part of Nvidia's deep learning SDK (not that you'll need to create an Nvidia developer account in order to download it). TensorFlow uses CUDA & cuDNN to control the GPU cards & accelerate computations.

<img src = "Images/TensorFlow uses CUDA & cuDNN to Control GPUs.png" width = "600" style = "margin:auto"/>

Once you have installed the GPU card(s) & all the required drivers & libraries, you can use the `nvidia-smi` command to check that CUDA is properly installed. It lists the available GPU cards, as well as processes running on each card:

```
nvidia-smi
```

At the time of this writing, you'll also need to install the GPU version of TensorFlow (i.e., the `tensorflow-gpu` library); however, there is ongoing work to have a unified installation procedure for both CPU-only & GPU machines, so please check the installation documentation to see which library you should install. In any case, since installing every required library correctly is a bit long & tricky (& all hell breaks loose if you do not install the correct library versions), TensorFlow provides a docker image with everything you need inside. However, in order for the docker container to have access to the GPU, you will still need to install the nvidia drivers on the host machine.

To check that TensorFlow actually sees the GPus, run the following tests:

In [ ]:
import tensorflow as tf

tf.test.is_gpu_available()

In [ ]:
tf.test.gpu_device_name()

In [ ]:
tf.config.experimental.list_physical_devices(device_type = "GPU")

This `is_gpu_available()` function checks whether at least one GPU is available. The `gpu_device_name()` function gives the first GPU's name: by default, operations will run on this GPU. The `list_physical_devices()` function returns the list of all available GPU devices (just one in this example).

Now, what if you don't want to invest time & money in getting your own GPU card? Just use GPU VM on the cloud!

## Using a GPU-Equipped Virtual Machine

All major cloud platforms now offer GPU VMs, some preconfigured with all the drivers & libraries you need (including TensorFlow). Google Cloud Platform enforces various GPU quotas, both worldwide & per region: you cannot just create thousands of GPU VMs without prior authorisation from Google. By default, the worldwide GPU quota is zero, so you cannot use any GPU VMs. Therefore, the very first thing you need to do is to request a higher worldwide quota. In the GCP console, open the navigation menu & go to IAM & admin -> Quotas. Click Metric, click None to uncheck all locations, then search for "GPU" & select "GPUs (all regions)" to see the corresponding quota. If this quota's value is zero (or just insufficient for your needs), then check the box next to it (it should be the only selected one) & click "Edit quotas". Fill in the requested information, then click "Submit request." It may take a few hours (or up to a few days) for your quota request to be processed & (generally) accepted. By default, there is also a quota of one GPU per region & per GPU type. You can request to increase the quotas too: click Metric, select None to uncheck all metrics, & click the location you want; check the boxes next to the quota(s) you want to change, & click "Edit quotas" to file a request.

Once your GPU quota requests are approved, you can in no time create a VM equipped with one or more GPUs by using Google Cloud AI Platform's *Depp Learning VM Images*: go to *https://homl.info/dlvm*, click View Console, then click "Launch on Compute Engine" & fill in the VM configuration form. Note that some locations do not have all types of GPUs, & some have no GPUs at all (change the location to see the types of GPUs available, if any). Make sure to select TensorFlow 2.0 as the framework, & check "Install NVIDIA GPU driver automaticaly on first startup". It is also a good idea to check "Enable access to JupyterLab via URL instead of SSH": this will make it very easy to start a Jupyter notebook running on this GPU VM, powered by JupyterLab (this is an alternative web interface to run Jupyter notebooks). Once the VM is created, scroll down the navigation menu to the artificial intelligence section, then click AI platform -> Notebooks. Once the notebook instance appears in the list (this may take a few minutes, so click Refresh once in a while until it appears), click its Open JupyterLab link. This will run JupyterLab on the VM & connect your browser to it. You can create notebooks & run any code you want on this VM, & benefit from its GPUs! 

But if you want to run some quick tests or easily share notebooks with your colleagues, then you should try Colaboratory.

## Colaboratory

The simplest & cheapest way to access a GPU VM is to use *Colaboratory* (or *Colab* for short). It's free! Just go to *https://colab.research.google.com/* & create a new Python 3 notebook: this will create a Jupyter notebook, stored on your Google Drive (alternatively, you can open any notebook on GitHub, or on Google Drive, or you can even upload your own notebooks). Colab's user interface is similar to Jupyter's, except you can share & use the notebooks like regular Google Docs, & there are a few other minor differences (e.g., you can create handy weights using special comments in your code).

When you open a Colab notebook, it runs on a free Google VM dedicated to that notebook, called a *Colab Runtime*. By default, the runtime is CPU-only, but you can change this by going to Runtime -> "Change runtime type", selecting GPU in the "Hardware accelerator" drop-down menu, then clicking Save. In fact, you can even select TPU! (Yes, you can actually use a TPU for free; we will talk about TPUs later in this chapter, though, for now just select GPU.)

<img src = "Images/Colab Runtimes & Notebooks.png" width = "600" style = "margin:auto"/>

Colab does have some restrictions: first, there is a limit to the number of Colab notebooks you can run simultaneously (currently 5 per Runtime type). Moreover, as the FAQ states, "Colaboratory is intended for interactive use. Long-running background computations, particularly on GPUs, may be stopped. Please do not use Colaboratory for cryptocurrency mining". Also, the web interface will automatically disconnect from the Colab Runtime if you leave it unattended for a while (~ 30 minutes). When you reconnect to the Colab Runtime, it may have been reset, so make sure you always export any data you care about (e.g., download it or save it to the Google Drive). Even if you never disconnect, the Colab Runtime will automatically shut down after 12 hours, as it is not meant for long-running computations. Despite these limitations, it's a fantastic tool to run tests easily, get quick results, & collaborate with your colleagues.

## Managing the GPU RAM

By default, TensorFlow automatically grabs all the RAM in all available GPUs the first time you run a computation. It does this to limit GPU RAM fragmentation. This means that if you try to start a second TensorFlow program (or any program that requires the GPU), it will quickly run out of RAM. This does not happen as often as you might think: usually a training script, a TF serving node, or a Jupyter notebook. If you need to run multiple programs for some reason (e.g., to train two different models in parallel on the same machine), then you will ned to split the GPU RAM between these processes more evenly.

If you have multiple GPU cards on your machine, a simple solution is to assign each of them to a single process. To do this, you can set the `CUDA_VISIBLE_DEVICES` environment variable so that each process only sees the appropriate GPU card(s). Also let the `CUDA_DEVICE_ORDER` environment variable to `PCI_BUS_ID` to ensure that each ID always refers to teh same GPU card. For example, if you have four GPU cards, you could start two programs, assigning two GPUs to each of them, by executing commands like the following in two seperate terminal windows:

```
CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=0,1 python3 program_1.py
# in another terminal:
CUDA_DEVICE_ORDER=PCI_BUS_ID CUDA_VISIBLE_DEVICES=3,2 python3 program_2.py
```

Program 1 will then only see GPU cards 0 & 1, named `/gpu:0` & `/gpu:1` respectively, & program 2 will only see GPU cards 2 & 3, name `/gpu:1` & `/gpu:0` respectively (note the order). Everything will work fine. Of course, you can also define these environment variables in Python by setting `os.environ["CUDA_DEVICE_ORDER"]` & `os.environ["CUDA_VISIBLE_DEVICES"]`, as long as you do so before using TensorFlow.

<img src = "Images/Each Program Gets Two GPUs.png" width = "550" style = "margin:auto"/>

Another option is to tell TensorFlow to grab only a specific amount of GPU RAM. This must be done immediately after importantly TensorFlow. For example, to make TensorFlow grab only 2GiB of RAM on each GPU, you must create  *virtual GPU device* (also called a *logical GPU device*) for each physical GPU device & set its memory limit to 2GiB (i.e., 2,048 MiB):

In [ ]:
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_virtual_device_configuration(
        gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 2048)]
    )

Now (supposing you have four GPUs, each with at least 4GiB of RAM) two programs like this one can run in parallel, each using all four GPU cards.

<img src = "Images/Each Program Gets Four GPUs, But Only 2GiB of RAM on Each GPU.png" width = "500" style = "margin:auto"/>

If you run the `nvidia-smi` command while both programs are running you should see that each process holds 2GiB of RAM on each card:

```
nvidia-smi
```

Yet another option is to tell TensorFlow to grab memory only when it needs it (this also must be done immediately after importing TensorFlow):

In [ ]:
for gpu in tf.config.experimental.list_physical_devices("GPU"):
    tf.config.experimental.set_memry_growth(gpu, True)

Another way to do this is to set the `TF_FORCE_GPU_ALLOW_GROWTH` environment variable to `True`. With this option, TensorFlow will never release memory once it has grabbed it (again, to avoid memory fragmentation), except of course when the program ends. It can be harder to guarantee deterministic behavior using this option (e.g., one program may crash because another program's memory usage went through the roof), so in production you'll probably want to stick with one of the previous options. However, there are some cases where it is very useful: for example, when you use a machine to run multiple Jupyter notebooks, several of which use TensorFlow. This is why the `TF_FORCE_GPU_ALLOW_GROWTH` environment variable is set to `True` in Colab Runtimes.

Lastly, in some cases you may want to split a GPU into two or more *virtual GPUs* -- for example, if you want to test a distribution algorithm (this is a handy way to try out the code examples in the rest of the lesson even if you have a single GPU, such as in a Colab Runtime). The following code splits the first GPU into two virtual devices, with 2GiB of RAM each (again, this must be done immediately after importing TensorFlow):

In [ ]:
physical_gpus = tf.config.experimental.list_physical_devices("GPU")
tf.config.experimental.set_virtual_device_configuration(
    physical_gpus[0],
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 2048),
     tf.config.experimental.VirtualDeviceConfiguration(memory_limit = 2048)]
)

These two virtual devices will then be called `/gpu:0` & `/gpu:1`, & you can place operations & variables on each of them as if they were really two independent GPUs. Now let's see how TensorFlow devices which devices it should place variables & execute operations on.

## Placing Operations & Variables on Devices

The TensorFlow whitepaper presents a friendly *dynamic placer* algorithm that automagically distributes operations across all available devices, taking into account times like the measured computation time in previous runs of the graph, estimations of the size of the input & output tensors for each operation, the amount of RAM available in each device, communication delay when transferring data into & out of devices, & hints & constraints from the user. In practice, this algorithm, turned out to be less efficient than a small set of placement rules specified by the user, so the TensorFlow team ended up dropping the dynamic placer.

That said, tf.keras & tf.data generally do a good job of placing operations & variables where they belong (e.g., heavy computations on the GPU, & data preprocessing on the CPU). But you can also place operations & variables manually on each device, if you want more control:

* As just mentioned, you generally want to place the data preprocessing operations on the CPU, & place the neural network operations on the GPUs.
* GPUs usually have a fairly limited communication bandwidth, so it is important to avoid unnecessary data transfers in & out of the GPUs.
* Adding more CPU RAM to a machine is simple & fairly cheap, so there's usually plenty of it, whereas the GPU RAM is baked into the GPU: it is an expensive & thus limited resource, so if a variable is not needed in the next few training steps, it should probably be placed on the CPU (e.g., datasets generally belong on the CPU).

You can safely ignore the prefix `/job:localhost/replica:0/task:0` for now (it allows you to place operations on other machines when using a TensorFlow cluster; we will talk about jobs, replicas, & tasks later in this chapter). As you can see, the first variable was placed on GPU 0, which is the default device. However, the second variable was placed on the CPU: this is because there are no GPU kernels for integer variables (or for operations involving integer tensors), so TensorFlow fell back to the CPU.

If you want to place an operation on a different device than the default one, use a `tf.device()` context:

In [ ]:
with tf.device("/cpu:0"):
    c = tf.Variable(42.0)

c.device

If you explicitly try to place an operation on variable on a device that does not exist or for which there is no kernel, then you will get an exception. However, in some cases you may prefer to fall back to the CPU; for example, if your program may run both on CPU-only machines & on GPU machines, you may want TensorFlow to ignore your `tf.device("/gpu:*"`) on CPU-only machines. To do this, you can call `tf.config.set_soft_device_placement(True)` just after importing TensorFlow: when a placement request fails, TensorFlow will fall back to its default placement rules (i.e., GPU 0 by default if it exists & there is a GPU kernel, & CPU 0 otherwise).

Now how exactly will TensorFlow execute all these operations across multiple devices?

## Parallel Execution Across Multiple Devices

As we've seen before, one of the benefits of using TF functions is parallelism. Let's look at this more closely. When TensorFlow runs a TF function, it starts by analysing its graph to find the list of operations that need to be evaluated, & it counts how many dependencies each of them has. TensorFlow then adds each operation with zero dependencies (i.e., each source operation) to the evaluation queue of this operation's device. Once an operation has been evaluated, the dependency counter of each operation that depends on it is decremented. Once an operation's dependency counter reacher zero, it is pushed to the evaluation queue of its device. Once all the nocde that TensorFlow needs have been evaluated, it returns their outputs.

<img src = "Images/Parallelised Execution of TensorFlow Graph.png" width = "600" style = "margin:auto"/>

Operations in the CPU's evaluation queue are dispatched to a thread pool called the *inter-op thread pool*. If the CPU has multiple corse, then these operations will effectively be evaluated in parallel. Some operations have multithreaded CPU kernels:
these kernels split their tasks into multiple suboperations, which are placed in another evaluation queue & dispatched to a second thread pool called the *intra-op thread pool* (shared by all multithreaded CPU kernels). In short, multiple operations & suboperations may be evaluated in parallel on different CPU cores.

For the GPU, things are a bit simpler. Operations in a GPU's evaluation queue are evaluated sequentially. However, most operations have multithreaded GPU kerenels, typically implemented by libraries that TensorFlow depends on, such as CUDA & cuDNN. These implementations have their own thread pools, & they typically exploit as many GPU threads as they can (which is the reason why there is no need for an inter-op thread pool in GPUs: each operation already floods most GPU threads).

For example, in the above figure, operations A, B, & C are source ops, so they can immediately be evaluated. Operations A & B are placed on teh CPU, so they are sent to the CPU's evaluation queue, then they are dispatched to the inter-op thread pool & immediately evaluated in parallel. Operation A happens to have a multi-threaded kernel; its computations are split into three parts, which are executed in parallel by the intra-op thread pool. Operation C goes to GPU 0's evaluation queue, & in this exmaple its GPU kernel happens to use cuDNN, which manages its won intra-op thread pool & runs the operation across many GPU threads in parallel. Suppose C finishes first. The dependency counters of D & E are decremented & they reach zero, so both operations are pushed to GPU 0's evaluation queue, & they are executed sequentially. Note that C only gets evaluated once, even though both D & E depend on it. Suppose B finishes next. Then F's dependency counter is decremented from 4 to 3, & since that's not 0, it does not run yet. Once A, D, & E are finished, then F's dependency counter reaches 0, & it is pushed to the CPU's evaluation queue & evaluated. Finally, TensorFlow returns the requested outputs.

An extra bit of magic that TensorFlow performs is when the TF function modifies a stateful resource, such as a variable: it ensures that the order of execution matches the order in the code, even if there is no explicit dependency between the statements. For example, if the TF function contains `v.assign_add(1)` followed by `v.assign(v * 2)`, TensorFlow will ensure that these operations are executed in that order.

With that, you have all you need to run any operation on any device, & exploit the power of your GPUs! Here are some of the things you could do:

* You could train several models in parallel, each on its own GPU: just write a training script for each model & run them in parallel, setting `CUDA_DEVICE_ORDER` & `CUDA_VISIBLE_DEVICES` so that each script only sees a single GPU device. This is great for hyperparameter tuning, as you can train in parallel multiple models with different hyperparameters. If you have a single machine with two GPUs, & it takes one hour to train one model on one GPU, then training two models in parallel, each on its own dedicated GPU, will take just one hour. Simple!
* You could train a model on a single GPU & perform all the preprocessing in parallel on the CPU, using the dataset's `prefetch()` method to prepare the next few batches in advance so they are ready when the GPU needs them.
* If your model takes two images an input & preprocesses them using two CNns before joining their outputs, then it will probably run much faster if you place each CNN on a different GPU.
* You can create an efficient ensemble: just place a different trained model on each GPu so that you can get all the predictions much faster to produce the emsemble's final prediction.

But what if you want to *train* a single model across multiple GPUs?

---

# Training Models Across Multiple Devices

There are two main approaches to training a single model across multiple devices: *model parallelism*, where the model is split across the devices, & *data parallelism*, where the model is replicated across every device, & each replica is trained on a subset of the data. Let's look at these two options closely before we train a model on multiple GPus.

## Model Parallelism

So far, we have trained each neural network on a single device. What if we want to train a single neural network across multiple devices? This requires chopping the model into several chunks & running each chunk on a different device. Unfortunately, such model parallelism turns out to be pretty tricky, & it really depends on the architecture of your neural network. For fully connected networks, there is generally not much to be gained from this approach.

<img src = "Images/Splitting a Fully Connected Neural Network.png" width = "600" style = "margin:auto"/>

Intuitively, it may seem that an easy way to split the model is to place each layer on a different device, but this does not work because each layer needs to wait for the output of the previous layer before it can do anything. So perhaps you can slice it vertically -- for example, with the left half of each layer on one device, & the right part on another device? This is slightly better, since both halves of each layer requires the output of both halves, so there will be a lot of cross-device communication (represented by the dashed arrows). This is likely to completely cancel out the benefit of the parallel computation, since cross-device communication is slow (especially when the devices are located on different machines).

Some neural network architecture, such as convolutional neural networks, contain layers that are only partially connected to the lower layers, so it is much easier to distribute chunks across devices in an efficient way.

<img src = "Images/Splitting Partially Connected Neural Network.png" width = "550" style = "margin:auto"/>

Deep recurrent neural networks can be split a bit more efficiently across multiple GPUs. If you split the network horizontally by placing each layer on a different device, & you feed the network with an input sequence to process, then at the first time step only one device will be active (working on the sequence's first value), at the second step two will be active (the second layer will be handling the second value), & by the time the signal propagates to the output layer, all devices will be active simultaneously. There is still a lot of cross-device communication going on, but since each cell may be fairly complex, the benefit of running multiple cells in parallel may (in theory) outweigh the communication penalty. However, in practice, a regular stack of `LSTM` layers running on a single GPU actually runs much faster.

<img src = "Images/Splitting a Deep Recurrent Neural Network.png" width = "600" style = "margin:auto"/>

In short, model parallelism may speed up running or training some types of neural networks, but not all, & it requires special care & tuning, such as making sure that devices that need to communicate the most run on the same machine. Let's look at a much simpler & generally more efficient option: data parallelism.

## Data Parallelism

Another way to parallelise the training of a neural network is to replicate on every device & run each training step simultaneously on all replicas, using a different mini-batch for each. The gradients computed by each replica are then averaged, & the result is used to update the model parameters. This is called *data parallelism*. There are many variants of this idea, so let's look at the most important ones.

### Data Parallelism Using the Mirrored Strategy

Arguably the simplest approach is to completely mirror all the model parameters across all the GPus & always apply the exact same parameter updates on every GPU. This way, all replicas always remain perfectly identical. This is called the *mirrored strategy*, & it turns out to be quite efficient, especially when using a single machine.

<img src = "Images/Data Parallelism Using Mirrored Strategy.png" width = "600" style = "margin:auto"/>

The tricky part when using this approach is to efficiently compute the mean of all the gradients from all the GPUs & distribute the result across all the GPUs. This can be done using an *AllReduce* algorithm, a class of algorithms where multiple nodes collaborate to efficiently perform a reduce operation (such as computing the mean, sum, & max), while ensuring that all nodes obtain the same final result. Fortunately, there are off-the-shelf implementations of such algorithms, as we will see.

### Data Parallelism with Centralised Parameters

Another approach is to store the model parameters outside of the GPu devices performing the computations (called *workers*), for example on the CPU. In a distributed setup, you may place all the parameters on one or more CPU-only servers called *parameter servers*, whose only role is to host & update the parameters.

<img src = "Images/Data Parallelism with Centralised Parameters.png" width = "600" style = "margin:auto"/>

Whereas the mirrored strategy imposed synchronous weight updates across all GPUs, this centralised approach allows either synchronous as asynchronous updates. Let's see the pros & cons of both options.

### Synchronous Updates

With *synchronous updates*, the aggregator waits until all gradients are available before it computes the average gradients & passes them to the optimiser, which will update the model parameters. Once a replica has finished computing its gradients, it must wait for the parameters to be updated before it can proceed to the next mini-batch. The downside is that some devices may be slower than others, so all other devices will have to wait for them at every step. Moreover, the parameters will be copied to every device almost at the same time (immediately after the gradients are applied), which may saturate the parameter servers' bandwidth.

### Asynchronous Updates

With asynchronous updates, whenever a replica has finished computing the gradients, it immediately uses them to update the model parameters. There is no aggregation (it removes the "mean" step from the above figure & no synchronisation. Replicas work independently of other replicas. Since there is no waiting for the other replicas, this approach runs more training steps per minute. Moreover, although the parameters still need to be copied to every device at every step, this happens at different times for each replica, so the risk of bandwidth saturation is reduced.

Data parallelism with asynchronous updates is an attractive choice becuase of its simplicity, the absence of synchronisation delay, & a better use of the bandwidth. However, although it works reasonably well in practice, it is almost surprising that it works at all! Indeed, by the time a replica has finished computing the gradients based on some parameter values, these parameters will have been updated several times by other replicas (on average N - 1 times, if there are N replicas), & there is no guarantee that the computed gradients will still be pointing in the right direction. When gradients are severely out of date, they are called *stalte gradients*: they can slow down convergence, introducing noise & wobble effects (the learning curve may contain temporary oscillations), or they can even make the training algorithm diverge.

<img src = "Images/Stale Gradients Using Asynchronous Updates.png" width = "500" style = "margin:auto"/>

There are a few ways you can reduce the effect of stale gradients:

* Reduce the learning rate.
* Drop stale gradients or scale them down.
* Adjust the mini-batch size.
* Start at the first few epochs using just one replica (this is called the *warmup phase*). Stale gradients tend to be more damaging at the beginning of training, when gradients are typically large & the parameters have not settled into a valley of the cost function yet, so different replicas may push the parameters in quite different directions.

A paper published by the Google Brain team in 2016 benchmarked various approaches & found that using synchronous updates with a few spare replicas was more efficient than using asynchronous updates, not only converging faster but also producing a better model. However, this is still an active area of research, so you should not rule out asynchronous updates just yet.

### Bandwidth Saturation

Whether you use synchronous or asynchronous updates, data parallelism with centralised parameters still requires communicating the model parameters from the parameter servers to every replica at the beginning of each training step, & the gradients in the other direction at the end of each training step. Similarly, when using the mirrored strategy, the gradients produced by each GPU will need to be shared with every other GPU. Unfortunately, there always comes a point where adding an extra GPU will not improve performance at all because the time spent moving the data into & out of GPU RAM (& across the network in a distributed setup) will outweigh the speedup obtained by splitting the computation load. At that point, adding more GPUs will just worsen the bandwidth saturation & actually slow down training.

Saturation is more severe for large dense models, since they have a lot of parameters & gradients to transfer. It is less severe for small models (but the parallelisation gain is limited) & for large sparse models, where the gradients are typically mostly zeros & so can be communicated efficiently. Jeff Dean, initiator & lead of the Google Brain project, reported typical speedups of 25-40x when distributing computations across 50 GPUs for dense models, & a 300x speedup for sparse models trained across 500 GPUs. As you can see, sparse models really do scale better. here are a few concrete examples:

* Neural machine translation: 6x speedup on 8 GPUs
* Inception/ImageNet: 32x speedup on 50 GPUs
* RankBrain: 300x speedup on 500 GPUs

Beyond a few dozen GPUs for a dense model or few hundred GPUs for a sparse model, saturation kicks in & performance degrades. There is plenty of research going on to solve this problem (exploring peer-to-peer architectures rather than centralised parameter servers, using lossy model compression, optimising when & what the replicas need to communicate, & so on), so there will likely be a lot of progress in parallelising neural networks in the next few years.

In the meantime, to reduce the saturation problem, you probably want to use a few powerful GPUs rather than plenty of weak GPUs, & you should also group your GPUs on few & very well interconnected servers. You can also try dropping the float precision for 32 bits (`tf.float32`) to 16 bits (`tf.float16`). This will cut in half the amount of data to transfer, often without much impact on the convergence rate or the model's performance. Lastly, if you are using centralised parameters, you can shard (split) the parameters across multiple parameter servers: adding more parameter servers will reduce the network load on each serve & limit the risk of bandwidth saturation.

Ok, now let's train a model across multiple GPUs!

## Training at Scale Using the Distribution Strategies API

Many models can be trained quite well on a single GPU, or even on a CPU. But if training is too slow, you can try distributing it across multiple GPUs on the same machine. If that's still too slow, try using more powerful GPUs, or adding more GPUs to the machine. If your model performs heavy computations (such as large matrix multiplications), then it will run much faster on powerful GPUs, & you could even try to use TPUs on Google Cloud AI Platform, which is usually run even faster for such models. but if you can't fit any more GPUs on the same machine, & if TPus aren't for you (e.g., perhaps your model doesn't benefit much from TPUs, or perhaps you want to use your own hardware infrastructure), then you can try training it across several servers, each with multiple GPUs (if this is still not enough, as a last resort, you can try adding some model parallelism, but this requires a lot more effort). In this section, we will see how to train models at scale, starting with multiple GPUs on the same machine (or TPUs) & then moving on to multiple GPUs across multiple machines.

Luckily, TensorFlow comes with a very simple API that takes care of all the complexity for you: the *distribution strategies API*. To train a keras model across all available GPUs(on a single machine) using data parallelism with the mirrored strategy, create a `MirroredStrategy` object, call its `scope()` methodtto get a distribution context, & wrap the creation & compilation of your model inside that context. Then call the model's `fit()` method normally.

In [ ]:
distribution = tf.distribute.MirroredStrategy()

with distribution.scope():
    mirrored_model = keras.model.Sequential([...])
    mirrored_model.compile([...])

batch_size = 100 # must be divisible by the number of replicas
history = mirrored_model.fit(X_train, y_train, epochs = 10)

Under the hood, tf.keras is distribution-aware, so in this `MirroredStrategy` context, it knows that it must replicate all variables & operations across all available GPU devices. Note that the `fit()` method will automatically split each training batch across all the replicas, so its' important that the batch size be divisible by the number of replicas. That's all! Training will be significantly faster than using a single device, & the code change was really minimal.

Once you have finished training your model, you can use it to make predictions efficiently: call the `predict()` method, & it will automatically split the batch across all replicas, making predictions in parallel (again, the batch size must be divisible by the number of replicas). If you call the model's `save()` method, it will be saved as a regular model, *not* as a mirrored model with multiple replicas. So when you load it, it will run like a regular model, on a single device (by default GPU 0, or the CPU if there are no GPUs). If you want to load a model & run it on all available devices, you must call `keras.models.load_model()` within a distribution context:

In [ ]:
with distribution.scope():
    mirrored_model = keras.model.load_model("my_mnist_model.h5")

If you only want a subset of all the available GPU devices, you can past the list to the `MirroredStrategy`'s constructor:

In [ ]:
distribution = tf.distribute.MirroredStrategy(["/gpu:0", "/gpu:1"])

By default, the `MirroredStrategy` class uses the *NVIDIA Collective Communications Library* (NCCL) for the AllReduce mean operation, but you can change it by setting the `cross_device_ops` argument to an instance of the `tf.distribution.HierarchicalCopyAllReduce` class, or an instance of the `tf.distribute.ReduceToOneDevice` class. The default NCCL option is based on the `tf.distribute.NcclAllReduce` class, which is usually faster, but this depends on the number & types of GPUs, so you may want to give the alternatives a try.

If you want to try using data parallelism with centralised parameters, replace the `MirroredStrategy` with the `CentralStorageStrategy`:

In [ ]:
distribution = tf.distribute.experimental.CentralStorageStrategy()

You can optionally set the `compute_devices` argument to specify the list of devices you want to use as workers (by default it will use all available GPUs), & you can optionally set the `parameter_device` argument to specify the device you want to store the parameters on (by default, it will use the CPU, or the GPU if there is just one).

Now let's see how to train a model across a cluster of TensorFlow servers!

## Training a Model on a TensorFlow Cluster

A *TensorFlow cluster* is a group fo TensorFlow processes running in parallel, usually on different machines, & talking to each other to complete some work -- for example, training or executing a neural network. Each TF process in the cluster is called a *task* or a *TF server*. It has an IP address, a port, & a type (also called its *role* or its *job*). The type can be either `"worker"`, `"chief"`, `"ps"` (parameter server), or `"evaluator"`:

* Each *worker* performs computations, usually on a machine with one or more GPUs.
* The *chief* performs computations as well (it is a worker), but it also handles extra work such as writing tensorfboard logs or saving checkpoints. There is a single chief in a cluster. If no chief is specified, then the first worker is the chief.
* A *parameter server* only keeps track of variable values, & it is usually on a CPU-only machine. This type of task is only used with the `ParameterServerStrategy`.
* An *evaluator* obviously takes care of evaluation.

To start a TensorFlow cluster, you must first specify it. This means defining each task's IP address, TCP port, & type. For example, the following *cluster specification* defines a cluster with three tasks (two workers & one parameter server). The cluster spec is a dictionary with one key per job, & the values are lists of task addresses (*IP:port*):

In [ ]:
cluster_spec = {
    "worker": ["machine-a.example.com:2222",  # /job:worker/task:0
               "machine-b.example.com:2222"], # /job:worker/task:1
    "ps": ["machine-a.example.com:2221"]      # /job:worker/task:0
}

<img src = "Images/TensorFlow Cluster.png" width = "600" style = "margin:auto"/>

In general, there will be a single task per machine, but as this example shows, you can configure multiple tasks on the same machine if you want (if they share the same GPUs, make sure the RAM is split up appropriately).

When you start a task, you must give it the cluster spec, & you must also tell it what its type & index are (e.g., worker 0). The simplest way to specify everything at once (both the cluster spec & the current task's type & index) is to set the `TF_CONFIG` environment variable before starting TensorFlow. It must be a JSON-encoded dictionary containing a cluster specification (under the `"cluster"` key) & the type & index of the current task (under the `"task"` key). For example, the following `TF_CONFIG` environment variable uses the cluster we just defined & specifies that the task to start is the first worker:

In [ ]:
import os, json

os.environ["TF_CONFIG"] = json.dumps({
    "cluster": cluster_spec,
    "task": {"type": "worker", "index": 0}
})

Now let's train a model on a cluster! We wil start with the mirrored strategy -- it's surprisingly simple! First, you need to set the `TF_CONFIG` environment variable appropriately for each task. There should be no parameter server (remove the "ps" key in the cluster spec), & in general you will want a single worker per machine. Make extra sure you set a different task index for each task. Finally, run the following training code on every worker:

In [ ]:
distribution = tf.distribute.experimental.MultiWorkerMirroredStrategy()

with distribution.scope():
    mirrored_model = keras.models.Sequential([...])
    mirrored_model.compile([...])

batch_size = 100 # must be divisible by the number of replicas
history = mirrored_model.fit(X_train, y_train, epochs = 10)

Yes, that's exactly the same code we used earlier, except this time we are using the `MultiWorkerMirroredStrategy` (in future versions, the `MirroredStrategy` will probably handle both the single machine & multimachine cases). When you start this script on the first workers, they will remain unblocked at the AllReduce step, but as soon as the last worker starts up training will begin, & you will see them all advancing at exactly the same rate (since they synchronise at each step).

You can also choose from two AllReduce implementations for this distribution strategy: a ring AllReduce algorithm based on gRPC for the network communications, & NCCL's implementation. The best algorithm to use depends on the number of workers, the number & types of GPUs, & the network. By default, TensorFlow will apply some heuristics to select the right algorithm for you, but if you want to force one algorithm, pass `CollectiveCommunication.RING` or `CollectiveCommunication.NCCL` (from `tf.distribute.experimental`) to the strategy's constructor.

If you prefer to implement asynchronous data parallelism with parameter servers, change the strategy to `ParameterServerStrategy`, add one or more parameter servers, & configure `TF_CONFIG` appropriately for each task. Note that although the workers will work asynchronously, the replicas on each worker will work synchronously.

Lastly, if you have access to TPUs on Google Cloud, you can create a `TPUStrategy` like this (then use it like the other strategies):

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver()
tf.tpu.experimental.initialise_tpu_system(resolver)
tpu_strategy = tf.distribute.experimental.TPUStrategy(resolver)

You can now train models across multiple GPUs & multiple servers: give yourself a pat on the back! If you want to train a large model, you will need many GPUs, across many servers, which will require either buying a lot of hardware or managing a lot of cloud VMs. In many cases, it's going to be less hassle & less expensive to use a cloud service that takes care of provisioning & managing all this infrastructure when you need it. Let's see how to do that on GCP.

## Running Large Training Jobs on Google Cloud AI Platform

If you decide to use Google AI Platform, you can deploy a training job with the same training code as you would run on your own TF cluster, & the platform will take care of provisioning & configuring as many GPU VMs as you desire (within your quotas).

To start the job, you will need the `gcloud` command-line tool, which is part of the Google Cloud SDK. You can either install the SDK on your own machine, or just use the Google Cloud Shell on GCP. This is a terminal you can use directly in your web browser; it runs on a free Linux VM (Debian), with the SDK already installed & preconfigured for you. The cloud shell is available anywhere in GCP: just click the Activate Cloud Shell icon ont eh top right of the page.

<img src = "Images/Activating Google Cloud Shell.png" width = "600" style = "margin:auto"/>

If you prefer to install the SDK on your machine, once you have installed it, you need to initialise it by running `gcloud init`: you will need to log in to GCP & grant access to your GCP resources, then select the GCP project you want to use (if you have more than one), as well as the region where you want the job to run. The `gcloud` command gives you access to every GCP feature, including the ones we used earlier. You don't have to go through the web interface every time; you can write scripts that start or stop VMs for you, deploy models, or perform any other GCP action.

Before you can run the training job, you needto write the training code, exactly like you did earlier for a distributed setup (e.g., using the `ParameterServerStrategy`). AI Platform will take care of setting `TF_CONFIG` for you on each VM. One that's done, you can deploy it & run it on a TF cluster with a command line like this:

```
gcloud ai-platform jobs submit training my_job_20241019_010300 \
    --region asia-souteast1 \
    --scale-tier PREMIUM_1 \
    --runtime-version 2.0 \
    --python-version 3.9 \
    --package-path /my_project/src/trainer \
    --module-name trainer.task \
    --staging-bucket gs://my-staging-bucket \
    --
    --my_extra_argument1 foo --my-extra-argument2 bar
```

Let's go through these options. The command will start a training job named `my_job_20241019-010300`, in the `asia-southeast1` region, using a `PREMIUM_1` *scale tier*: this corresponds to 20 workers (including a chief) & 11 parameter serves (check out the other available scale tiers). All these VMs will be based on AI platform's 2.0 runtime (a VM configuration that includes TensorFlow 2.0 & many other packages) & Python 3.9. The training code is lodated in the */my_project/src/trainer* directory, & the `gcloud` command will automatically bundle it into a pip package & upload it to GCS at *gs://my-stagin-bucket*. Next, AI Platform will start everal VMs, deploy the package to them, & run the `trainer.task` module. Lastly, the `--job-dir` argument & the extra arguments (i.e., all the arguments located after the `--` separator) will be passed to the training program: the chief task will usually use the `--job-dir` argument to find out where to save the final model on GCS, in this case at *gs://my-mnist-model-bucket/trained_model*. Thats it! In the GCP console, you can then open the navigation menu, scroll down to the Artificial Intelligence section, & open AI Platform -> Jobs. You should see your job running, & if you click it you will see graphs showing the CPU, GPU, & RAM utilisation for every task. You can click View Logs to access the detailed logs using Stackdriver.

If you want to explore a few hyperparameter values, you can simply run multiple jobs & specify the hyperparameter values using the extra arguments for your tasks. however, if you want to explore many hyperparameters efficiently, it's a good idea to use AI Platform's hyperparameter tuning service instead.

## Black Box Hyperparameter Tuning on AI Platform

AI Platform provides a powerful Bayesian optimisation hyperparameter tuning service called Google Vizier. TO use it, you need to pass a YAML configuration file when creating the job (`--config tuning.yaml`). For example, it may look like this:

```
trainingInput:
    hyperparameters:
        goal: MAXIMIZE
        hyperparameterMetricTag: accuracy
        maxTrials: 10
        maxParallelTrials: 2
        params:
            - parameterName: n_layers
              type: INTEGER
              minValue: 10
              maxValue: 100
              scaleType: UNIT_LINEAR_SCALE
            - parameterName: momentum
              type: DOUBLE
              minValue: 0.1
              maxValue: 1.0
              scaleType: UNIT_LOG_SCALE
```

This tells AI Platform that we want to maximise the metric named `"accuracy"`, the job will run a maximum of 10 trials (each trial will run our training code to train the model from scratch), & it will run a maximum of 2 trials in parallel. We want it to tune two hyperparameters: the `n_layers` hyperparameter (an integer between 10 & 100) & the `momentum` hyperparameter (a float between 0.1 & 1.0). The `scaleType` argument specifies the prior for the hyperparameter value: `UNIT_LINEAR_SCALE` means a flat prior (i.e., no a prior preference), while `UNIT_LOG_SCALE` says we have a prior belief that the optimal value lies closer to the max value (the other possible prio is `UNIT_REVERSE_LOG_SCALE`, when we believe the optimal value to be close to the min value).

The `n_layers` & `momentum` arguments will be passed as command-line arguments to the training code, & of course it is expected to use them. The question is, how will the training code communicate the metric back to the AI Platform so that it can decide which hyperparameter values to use during the next trial? Well, AI Platform just monitors the output directory (specified via `--job-dir`) for any event file containing summarise for a metric name `"accuracy"` (or whatever metric name is specified as the `hyperparameterMetricTag`), & it reads those values. So your training code simply has to use the `TensorBoard()` callback (which you will want to do anyway for monitoring), & you're good to go!

Once the job is finished, all the hyperparameter values used in each trial & the resulting accuracy will be available in the job's output (available via the AI Platform -> Jobs page).

Now you have all the tools & knowledge you need to create state-of-the-art neural net architectures & train them at scale using various distribution strategies, on your own infrastructure or on the cloud -- & you can even perform powerful Bayesian optimisation to fine-tune the hyperparameters!